# Generar mapa de surditoras de plaguicidas

In [16]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

## Localización y carga de los datos 

In [6]:
plag = pd.read_excel("data/denue/DENUE_52_Plaguicidas_1.xlsx", sheet_name="DENUE_52_Plaguicidas_1")
plag.head()

,ID,NOM_ESTAB,RAZ_SOCIAL,CODIGO_ACT,NOMBRE_ACT,PER_OCU,TIPO_VIAL,NOM_VIAL,TIPO_V_E_1,NOM_V_E_1,...,WWW,TIPOUNIECO,LATITUD,LONGITUD,FECHA_ALTA,OID_1,BUFER_POS,RADIO_AFEC,CLAVE_1,OID_12
0,6256708,DESTILADORA DE ACEITE DE LIMbN,CITRO JUGO SA DE CV,325999,FabricaciOn de otros productos qubmicos,51 a 100 personas,CARRETERA,CARRETERA FEDERAL LIBRE 200 TRAMO TECOM,NaN,NINGUNO,...,WWW.CITROJUGO.COM,Fijo,18.889444,-103.853333,DICIEMBRE 2014,142,0,1500,AII-CII,2
1,333271,COMERCIALIZADORA AGRlCOLA,COMERCIALIZADORA AGRlCOLA,434111,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,BOULEVARD,RODOLFO CHoVEZ CARRILLO,CALLE,FRAY PEDRO DE GANTE,...,NaN,Fijo,19.244059,-103.736456,DICIEMBRE 2014,534,2,15000,AI-CIII,11
2,6695545,FERTILIZANTES TEPEYAC BODEGA KASOMA MANZANILLO,"FERTILIZANTES TEPEYAC, S.A. DE C.V.",434111,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,CARRETERA,CARRET. A MINATITLAN,OTRA (ESPECIFICAR),NINGUNO,...,NaN,Fijo,19.105278,-104.266389,JULIO 2010,808,2,15000,AI-CIII,18
3,6303560,AGROSERVICIOS NACIONALES O ANSA,AGROSERVICIOS NACIONALES SAPI DE CV,434111,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,CALLE,JUAN DE LA BARRERA,CALLE,MANUEL ACU A,...,WWW.ANSA.CC,Fijo,18.919823,-103.880255,DICIEMBRE 2014,1272,2,15000,AI-CIII,30
4,6303737,FERMAN,FERTILIZANTES MANUFACTURADOS DE MEXICO S DE RL...,434111,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,AVENIDA,INSURGENTES,CALLE,NINGUNO,...,NaN,Fijo,18.938578,-103.893803,JULIO 2010,1336,2,15000,AI-CIII,32


In [7]:
plag.columns

Index(['ID', 'NOM_ESTAB', 'RAZ_SOCIAL', 'CODIGO_ACT', 'NOMBRE_ACT', 'PER_OCU',
       'TIPO_VIAL', 'NOM_VIAL', 'TIPO_V_E_1', 'NOM_V_E_1', 'TIPO_V_E_2',
       'NOM_V_E_2', 'TIPO_V_E_3', 'NOM_V_E_3', 'NUMERO_EXT', 'LETRA_EXT',
       'EDIFICIO', 'EDIFICIO_E', 'NUMERO_INT', 'LETRA_INT', 'TIPO_ASENT',
       'NOMB_ASENT', 'TIPOCENCOM', 'NOM_CENCOM', 'NUM_LOCAL', 'COD_POSTAL',
       'CVE_ENT', 'ENTIDAD', 'CVE_MUN', 'MUNICIPIO', 'CVE_LOC', 'LOCALIDAD',
       'AGEB', 'MANZANA', 'TELEFONO', 'CORREOELEC', 'WWW', 'TIPOUNIECO',
       'LATITUD', 'LONGITUD', 'FECHA_ALTA', 'OID_1', 'BUFER_POS', 'RADIO_AFEC',
       'CLAVE_1', 'OID_12'],
      dtype='object')

In [9]:
cols_to_keep = ['NOM_ESTAB', 'NOMBRE_ACT','PER_OCU','ENTIDAD', 'MUNICIPIO','LOCALIDAD', 'AGEB', 'LATITUD', 'LONGITUD']
plaguicidas = plag[cols_to_keep]
plaguicidas.head()

,NOM_ESTAB,NOMBRE_ACT,PER_OCU,ENTIDAD,MUNICIPIO,LOCALIDAD,AGEB,LATITUD,LONGITUD
0,DESTILADORA DE ACEITE DE LIMbN,FabricaciOn de otros productos qubmicos,51 a 100 personas,COLIMA,TECOMAN,Ninguno [Destiladora de Aceite de LimOn],0126,18.889444,-103.853333
1,COMERCIALIZADORA AGRlCOLA,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,COLIMA,COLIMA,Colima,0731,19.244059,-103.736456
2,FERTILIZANTES TEPEYAC BODEGA KASOMA MANZANILLO,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,COLIMA,MANZANILLO,Interlinks,1416,19.105278,-104.266389
3,AGROSERVICIOS NACIONALES O ANSA,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,COLIMA,TECOMAN,TecomAn,0450,18.919823,-103.880255
4,FERMAN,"Comercio al por mayor de fertilizantes, plagui...",0 a 5 personas,COLIMA,TECOMAN,TecomAn,0323,18.938578,-103.893803


In [30]:
rename_municipios = {
  "Tecoman": "Tecomán",
  "Minatitlan": "Minatitlán",
  "Cuauhtemoc": "Cuauhtémoc",
  'Villa de Alvarez':"Villa de Álvarez",
  'Ixtlahuacan':"Ixtlahuacán",
  'Coquimatlan':"Coquimatlán",
  'Armeraa':"Armería",
}

In [15]:
# set every value to Title Case
for col in cols_to_keep:
  try:
    plaguicidas[col] = plaguicidas[col].str.title()
  except:
    continue

plaguicidas['MUNICIPIO'] = plaguicidas['MUNICIPIO'].replace(rename_municipios)
plaguicidas['LOCALIDAD'] = plaguicidas['LOCALIDAD'].replace(rename_municipios)
plaguicidas.head()

C:\Users\pedro\AppData\Local\Temp\ipykernel_32192\3811444496.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plaguicidas[col] = plaguicidas[col].str.title()
C:\Users\pedro\AppData\Local\Temp\ipykernel_32192\3811444496.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plaguicidas['MUNICIPIO'] = plaguicidas['MUNICIPIO'].replace(rename_municipios)
C:\Users\pedro\AppData\Local\Temp\ipykernel_32192\3811444496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,NOM_ESTAB,NOMBRE_ACT,PER_OCU,ENTIDAD,MUNICIPIO,LOCALIDAD,AGEB,LATITUD,LONGITUD
0,Destiladora De Aceite De Limbn,Fabricacion De Otros Productos Qubmicos,51 A 100 Personas,Colima,Tecomán,Ninguno [Destiladora De Aceite De Limon],0126,18.889444,-103.853333
1,Comercializadora Agrlcola,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Colima,Colima,0731,19.244059,-103.736456
2,Fertilizantes Tepeyac Bodega Kasoma Manzanillo,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Manzanillo,Interlinks,1416,19.105278,-104.266389
3,Agroservicios Nacionales O Ansa,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Tecomán,Tecomán,0450,18.919823,-103.880255
4,Ferman,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Tecomán,Tecomán,0323,18.938578,-103.893803


In [18]:
gdf_plaguicidas = gpd.GeoDataFrame(plaguicidas, geometry=gpd.points_from_xy(plaguicidas.LONGITUD, plaguicidas.LATITUD), crs="EPSG:4326")
gdf_plaguicidas = gdf_plaguicidas.to_crs("EPSG:3857")
gdf_plaguicidas.drop(columns=['LATITUD', 'LONGITUD'], inplace=True)
gdf_plaguicidas.head()

,NOM_ESTAB,NOMBRE_ACT,PER_OCU,ENTIDAD,MUNICIPIO,LOCALIDAD,AGEB,geometry
0,Destiladora De Aceite De Limbn,Fabricacion De Otros Productos Qubmicos,51 A 100 Personas,Colima,Tecomán,Ninguno [Destiladora De Aceite De Limon],0126,POINT (-11560900.182 2141924.105)
1,Comercializadora Agrlcola,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Colima,Colima,0731,POINT (-11547889.423 2183691.133)
2,Fertilizantes Tepeyac Bodega Kasoma Manzanillo,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Manzanillo,Interlinks,1416,POINT (-11606881.318 2167334.597)
3,Agroservicios Nacionales O Ansa,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Tecomán,Tecomán,0450,POINT (-11563897.042 2145498.661)
4,Ferman,"Comercio Al Por Mayor De Fertilizantes, Plagui...",0 A 5 Personas,Colima,Tecomán,Tecomán,0323,POINT (-11565405.255 2147705.802)


In [19]:
# save to gpkg
gdf_plaguicidas.to_file("data/denue/plaguicidas.gpkg", driver="GPKG")


# load localidades

In [34]:
mun_cve = {
  '001' : 'Armería',
  '002' : 'Colima',
  '003' : 'Comala',
  '004' : 'Coquimatlán',
  '005' : 'Cuauhtémoc	',
  '006' : 'Ixtlahuacán',
  '007' : 'Manzanillo',
  '008' : 'Minatitlán',
  '009' : 'Tecomán',
  '010' : 'Villa de Álvarez'	
}

In [38]:
loc = gpd.read_file("data/denue/06l.shp")
loc = loc.to_crs("EPSG:3857")
# user mun_cve to create a new column with the municipio name based on CVE_MUN
loc['MUNICIPIO'] = loc['CVE_MUN'].map(mun_cve)

loc

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,NOMGEO,AMBITO,geometry,MUNICIPIO
0,060010032,06,001,0032,El Paraíso [Balneario],Rural,"POLYGON ((-11576698.578 2140685.392, -11576667...",Armería
1,060010341,06,001,0341,Colonia Flor de Coco,Rural,"POLYGON ((-11571107.175 2149335.819, -11571061...",Armería
2,060010034,06,001,0034,El Puertecito,Rural,"POLYGON ((-11568289.821 2159802.189, -11568343...",Armería
3,060010042,06,001,0042,Los Reyes (Zorrillos),Rural,"POLYGON ((-11584500.273 2153178.072, -11584464...",Armería
4,060010033,06,001,0033,Periquillo,Rural,"POLYGON ((-11571902.001 2150151.198, -11571881...",Armería
...,...,...,...,...,...,...,...,...
161,060100002,06,010,0002,Agua Dulce,Rural,"POLYGON ((-11564692.159 2189112.062, -11564717...",Villa de Álvarez
162,060100301,06,010,0301,Residencial Campestre Buenos Aires [Fraccionam...,Rural,"POLYGON ((-11547047.498 2192366.433, -11547093...",Villa de Álvarez
163,060100022,06,010,0022,El Mixcuate,Rural,"POLYGON ((-11570032.469 2192764.432, -11570056...",Villa de Álvarez
164,060100001,06,010,0001,Ciudad de Villa de Álvarez,Urbana,"MULTIPOLYGON (((-11551550.853 2184894.032, -11...",Villa de Álvarez


In [39]:
# save to gpkg
loc.to_file("data/denue/06l.gpkg", driver="GPKG")